In [ ]:
import numpy as np
import pandas as pd
import json
from erresire.create_lens_pop import CreateLensPop
from astropy.cosmology import FlatLambdaCDM


In [ ]:
#define cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.05)

This example demonstrates the use of built-in model functions: an elliptical NFW halo, and a galaxy represented by two Sérsic components (one for the disk and one for the bulge).

**Note**: The built-in functions are optimized for the catalogs used in Mezini 2025, but defining custom functions is straightforward. Custom components can be integrated seamlessly into the modeling pipeline.

create_lens_pop automatically searches for built-in component functions defined in create_lens_model.py. The currently supported built-in component types include:

"nfw_ellipse"

"sersic_ellipse_disk"

"sersic_ellipse_sphere"

As shown in the example, galaxies and halos may consist of multiple components. To construct a multi-component model, simply provide additional component types and their associated functions in the input list.

In [44]:
# number of lenses to simulate
n_lens = 50

# Load input configuration
with open("lens_model_input.json", "r") as f:
    cfg = json.load(f)

# Redshift limits
z_min = cfg["z_min"]
z_max = cfg["z_max"]

# Lens model components
halo_type = cfg["halo_type"]          # Example: ["NFW_ELLIPSE_POTENTIAL"]
halo_function = cfg["halo_function"]  # Example: ["nfw_ellipse"]

# Example: ["SERSIC_ELLIPSE_POTENTIAL", "SERSIC_ELLIPSE_POTENTIAL"]
galaxy_type = cfg["galaxy_type"]
# Example: ["sersic_ellipse_disk", "sersic_ellipse_sphere"]
galaxy_function = cfg["galaxy_function"]

# Load catalogs
halo_catalog = pd.read_csv(cfg["halo_catalog"])
galaxy_catalog = pd.read_csv(cfg["galaxy_catalog"])
source_catalog = pd.read_csv(cfg["source_catalog"])

# Include external shear?
shear = cfg["shear"]  # Boolean (True/False)

In [24]:
clp = CreateLensPop(cosmo=cosmo,
                    halo_catalog=halo_catalog,
                    galaxy_catalog=galaxy_catalog,
                    source_catalog=source_catalog,
                    halo_type=halo_type,
                    galaxy_type=galaxy_type,
                    galaxy_function=galaxy_function,
                    halo_function=halo_function,
                    halo_catalog_mass_key = 'halo_mass',
                    galaxy_catalog_halo_mass_key = 'halo_mass',
                    galaxy_catalog_redshift_key = 'redshift',
                    source_catalog_redshift_key = 'redshift',
                    shear=shear)

In [25]:
results = []

for _ in range(n_lens):
    results.append(
        clp.monte_carlo(z_lens_min=z_min, z_lens_max=z_max, z_source_max=4.0)
    )

df = pd.concat(results, ignore_index=True)


In [26]:
df

,n_img,s_id,source_DEC,source_RA,z_lens,z_source,caustic_radius,source_dist,lens_source_dist,caustic_area,...,center_x2,center_y2,e12,e22,Rs,alpha_Rs,e13,e23,center_x3,center_y3
0,1,669000,-1.334150,1.393444,0.438851,0.836068,2.5,2885.314372,1200.459260,0.188377,...,-0.010892,-0.019654,-0.012447,-0.010906,12.733147,0.562887,-0.071608,-0.062968,0.0,0.0
1,0,295000,1.042815,2.001426,0.438851,2.123627,2.5,5353.411962,3668.556850,0.647232,...,0.020461,0.058377,-0.012447,-0.010906,12.733147,0.927111,-0.071608,-0.062968,0.0,0.0
2,3,430000,-0.187977,0.347537,0.438851,1.683690,2.5,4687.272049,3002.416937,0.547457,...,0.064728,-0.002235,-0.012447,-0.010906,12.733147,0.866598,-0.071608,-0.062968,0.0,0.0
3,1,335000,-0.706524,0.538297,0.438851,1.093921,2.5,3526.200476,1841.345363,0.328872,...,0.076024,-0.001847,-0.012447,-0.010906,12.733147,0.706472,-0.071608,-0.062968,0.0,0.0
4,1,128000,-0.395013,0.658304,0.438851,2.239222,2.5,5507.245014,3822.389902,0.672120,...,-0.013955,-0.019378,-0.012447,-0.010906,12.733147,0.939004,-0.071608,-0.062968,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,574000,-0.218162,-1.749994,1.267425,2.534368,2.5,5867.231993,1960.185081,0.190930,...,0.003228,0.020890,0.000873,-0.003929,6.612735,0.772868,0.025043,-0.032872,0.0,0.0
246,1,242000,-0.088973,-0.526102,1.267425,1.525522,2.5,4410.634837,503.587924,0.012402,...,-0.041972,0.018205,0.000873,-0.003929,6.612735,0.264129,0.025043,-0.032872,0.0,0.0
247,3,301000,-0.037574,0.193361,1.267425,2.249145,2.5,5520.091739,1613.044826,0.133821,...,0.043533,-0.032076,0.000873,-0.003929,6.612735,0.675992,0.025043,-0.032872,0.0,0.0
248,1,386000,1.558137,-0.961570,1.267425,2.460665,2.5,5781.428662,1874.381749,0.175741,...,0.035028,-0.003579,0.000873,-0.003929,6.612735,0.750005,0.025043,-0.032872,0.0,0.0


**Example** Defining custom functions

You can also define your own component functions. The only requirement is that each custom function must produce outputs compatible with lenstronomy mass models.

In this case, the user specifies ``halo_function="CUSTOM_HALO"`` and then provides the name of the custom function, such as ``nfw_ellipse_create_kwargs`` in the example below.

The same approach applies to galaxy components: set ``galaxy_function="CUSTOM_GALAXY"`` and supply the corresponding custom function name.

In [32]:
import lenstronomy.Util.param_util as param_util


def nfw_ellipse_create_kwargs(LC, properties, x, y):
    """
    Custom NFW elliptical halo model.
    Must return kwargs compatible with lenstronomy.

    Args:
        LC: LensCosmo object from lenstronomy
        properties (pd.DataFrame): halo catalog row
        x, y (float): halo center

    Returns:
        dict: kwargs for lenstronomy NFW_ELLIPSE_POTENTIAL
    """
    c = properties["c"].values[0]
    m200 = properties["halo_mass"].values[0]

    Rs_angle, alpha_Rs = LC.nfw_physical2angle(M=m200, c=c)

    # Ellipticity
    q = properties["q"].values[0]
    phi = properties["position_angle"].values[0]
    e1, e2 = param_util.phi_q2_ellipticity(phi=phi, q=q)

    return {
        "Rs": Rs_angle,
        "alpha_Rs": alpha_Rs,
        "e1": e1,
        "e2": e2,
        "center_x": x,
        "center_y": y,
    }


In [45]:
# Number of lens systems to simulate
n_lens = 50

# Load configuration file
with open("lens_model_input_custom_function.json", "r") as f:
    cfg = json.load(f)

# Redshift limits for lens population
z_min = cfg["z_min"]
z_max = cfg["z_max"]

# Halo model specification
# Example values: ["NFW_ELLIPSE_POTENTIAL"]
halo_type = cfg["halo_type"]

# Example values: ["CUSTOM_HALO"] — tells CreateLensPop to call your custom halo function
halo_function = cfg["halo_function"]

# Galaxy model specification
# Example: ["SERSIC_ELLIPSE_POTENTIAL", ...]
galaxy_type = cfg["galaxy_type"]

# Example: ["sersic_ellipse_disk", "sersic_ellipse_sphere"]
galaxy_function = cfg["galaxy_function"]

# Input catalogs
halo_catalog = pd.read_csv(cfg["halo_catalog"])
galaxy_catalog = pd.read_csv(cfg["galaxy_catalog"])
source_catalog = pd.read_csv(cfg["source_catalog"])

# Whether to include external shear in the simulation
shear = cfg["shear"]

In [ ]:
clp = CreateLensPop(cosmo=cosmo,
                    halo_catalog=halo_catalog,
                    galaxy_catalog=galaxy_catalog,
                    source_catalog=source_catalog,
                    halo_type=halo_type,
                    halo_function=halo_function,
                    custom_halo_function=nfw_ellipse_create_kwargs,   # <<-- IMPORTANT
                    galaxy_type=galaxy_type,
                    galaxy_function=galaxy_function,
                    shear=shear)


In [35]:
results = []

for _ in range(n_lens):
    results.append(
        clp.monte_carlo(z_lens_min=z_min, z_lens_max=z_max, z_source_max=4.0)
    )

df = pd.concat(results, ignore_index=True)

In [36]:
df

,n_img,s_id,source_DEC,source_RA,z_lens,z_source,caustic_radius,source_dist,lens_source_dist,caustic_area,...,center_x2,center_y2,e12,e22,Rs,alpha_Rs,e13,e23,center_x3,center_y3
0,1,639000,-0.833363,1.486968,1.214491,2.161812,2.5,5405.096593,1610.344563,0.213474,...,-0.028096,0.019078,0.084940,0.089458,2.996697,0.217169,0.021741,0.036479,0.0,0.0
1,1,157000,0.730277,-0.061567,1.214491,2.238445,2.5,5506.237664,1711.485635,0.232322,...,0.006835,0.048293,0.084940,0.089458,2.996697,0.226569,0.021741,0.036479,0.0,0.0
2,1,539000,-0.557585,0.508059,1.214491,1.299393,2.5,3973.291088,178.539059,0.001945,...,0.006955,0.029169,0.084940,0.089458,2.996697,0.032754,0.021741,0.036479,0.0,0.0
3,1,340000,-0.127998,-0.695257,1.214491,2.204412,2.5,5461.738474,1666.986444,0.225407,...,-0.004520,-0.029766,0.084940,0.089458,2.996697,0.222476,0.021741,0.036479,0.0,0.0
4,1,518000,-1.003365,0.025999,1.214491,2.424856,2.5,5738.798753,1944.046724,0.289329,...,-0.020321,0.001935,0.084940,0.089458,2.996697,0.246927,0.021741,0.036479,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,248000,-1.305434,-1.678256,1.348003,2.279322,2.5,5558.824325,1486.996510,0.079561,...,-0.020107,-0.012846,0.011621,-0.001523,7.930944,0.957505,0.016460,0.004275,0.0,0.0
496,1,430000,-0.309648,0.355793,1.348003,1.683690,2.5,4687.272049,615.444233,0.010034,...,0.000237,0.012731,0.011621,-0.001523,7.930944,0.469983,0.016460,0.004275,0.0,0.0
497,1,428000,-0.891388,-1.112468,1.348003,2.337301,2.5,5631.850575,1560.022760,0.086406,...,-0.000558,-0.039683,0.011621,-0.001523,7.930944,0.991503,0.016460,0.004275,0.0,0.0
498,1,232000,-1.374627,0.723177,1.348003,2.189612,2.5,5442.180388,1370.352572,0.064875,...,0.005945,0.049865,0.011621,-0.001523,7.930944,0.901308,0.016460,0.004275,0.0,0.0


**Example** Galaxy catalog does not follow halo mass function.

If your galaxy catalog does not naturally follow the halo mass function, you can use the ``compute_hmf_weights()`` function prior to running the simulation. After computing these weights, pass them as input to CreateLensPop.

The function ``compute_hmf_weights()`` assigns each galaxy a probability weight based on its host halo mass.

In [ ]:
# number of lenses to simulate
n_lens = 50

# Load input configuration
with open("lens_model_input.json", "r") as f:
    cfg = json.load(f)

z_min = cfg["z_min"]
z_max = cfg["z_max"]

halo_type = cfg["halo_type"]
halo_function = cfg["halo_function"]

galaxy_type = cfg["galaxy_type"]
galaxy_function = cfg["galaxy_function"]

halo_catalog = pd.read_csv(cfg["halo_catalog"])
galaxy_catalog = pd.read_csv(cfg["galaxy_catalog"])
source_catalog = pd.read_csv(cfg["source_catalog"])


shear = cfg['shear']

In [37]:
from lens_model_extra_methods import compute_hmf_weights

weights =  compute_hmf_weights(galaxy_catalog, mass_key='halo_mass', redshift_key='redshift',
                        z_grid=np.linspace(0, 3, 50), num_mass_bins=250)

In [ ]:
clp = CreateLensPop(cosmo=cosmo,
                    halo_catalog=halo_catalog,
                    galaxy_catalog=galaxy_catalog,
                    source_catalog=source_catalog,
                    halo_type=halo_type,
                    halo_function=halo_function,
                    custom_halo_function=nfw_ellipse_create_kwargs,   # <<-- IMPORTANT
                    galaxy_type=galaxy_type,
                    galaxy_function=galaxy_function,
                    shear=shear,
                    mass_function_weights=weights)  # <------ use HMF weights here

In [39]:
results = []

for _ in range(n_lens):
    results.append(
        clp.monte_carlo(z_lens_min=z_min, z_lens_max=z_max, z_source_max=4.0)
    )

df = pd.concat(results, ignore_index=True)

In [40]:
df

,n_img,s_id,source_DEC,source_RA,z_lens,z_source,caustic_radius,source_dist,lens_source_dist,caustic_area,...,center_x2,center_y2,e12,e22,Rs,alpha_Rs,e13,e23,center_x3,center_y3
0,1,123000,-1.182191,2.053076,1.411242,1.721549,2.5,4750.280127,554.076689,0.008158,...,-0.023374,0.006000,-0.101769,0.030361,1.496649,0.094038,-0.020058,0.073741,0.0,0.0
1,1,473000,0.172309,-1.142584,1.411242,1.804127,2.5,4883.710585,687.507147,0.012628,...,-0.031695,0.021612,-0.101769,0.030361,1.496649,0.113496,-0.020058,0.073741,0.0,0.0
2,1,702000,-1.560550,0.659996,1.411242,1.625047,2.5,4587.288630,391.085192,0.003007,...,0.023741,0.026494,-0.101769,0.030361,1.496649,0.068733,-0.020058,0.073741,0.0,0.0
3,1,324000,-1.437957,0.469757,1.411242,2.617529,2.5,5961.036039,1764.832600,0.094342,...,0.008210,0.028240,-0.101769,0.030361,1.496649,0.238690,-0.020058,0.073741,0.0,0.0
4,1,212000,-0.252117,-1.423983,1.411242,1.772836,2.5,4833.782857,637.579418,0.011115,...,0.011051,-0.017284,-0.101769,0.030361,1.496649,0.106341,-0.020058,0.073741,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,370000,-1.983162,-0.515697,0.931986,1.144772,2.5,3641.697765,506.630391,0.001468,...,-0.033777,-0.022699,-0.162033,-0.072712,5.988433,0.306415,-0.057223,0.003347,0.0,0.0
496,1,748000,0.153822,-0.106202,0.931986,2.225206,2.5,5489.005618,2353.938245,0.082970,...,0.036217,-0.031125,-0.162033,-0.072712,5.988433,0.944549,-0.057223,0.003347,0.0,0.0
497,1,515000,-1.311915,0.887959,0.931986,2.460892,2.5,5781.697442,2646.630069,0.097344,...,0.047861,0.013487,-0.162033,-0.072712,5.988433,1.008234,-0.057223,0.003347,0.0,0.0
498,1,407000,-0.868733,-0.589626,0.931986,1.597181,2.5,4538.729593,1403.662220,0.023214,...,0.067609,0.018420,-0.162033,-0.072712,5.988433,0.681164,-0.057223,0.003347,0.0,0.0
